<a href="https://colab.research.google.com/github/anasali0006/Tutorial-Yolov4-Object-Detection-with-Custom-Dataset/blob/main/Yolov4_training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Mount with your Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

## Setup darknet environment

In [ ]:
# check whether GPU is provided
!nvcc --version


In [ ]:
import os
assert os.getcwd()=='/content', 'Directory should be "/content" instead of "{}"'.format(os.getcwd())

# remove the existing folder if have
#!rm -r darknet_for_colab

# download and compile darknet_for_colab
!git clone https://github.com/anasali0006/darknet_for_colab.git
%cd darknet_for_colab
!make
!chmod +x ./darknet

## Download yolov4 pre-trained weights

In [ ]:
assert os.getcwd()=='/content/darknet_for_colab', 'Directory should be "/content/darknet_for_colab" instead of "{}"'.format(os.getcwd())
!wget https://github.com/AlexeyAB/darknet/releases/download/darknet_yolo_v3_optimal/yolov4.conv.137

## Download custom dataset (yolo format)

In [ ]:
%cd data
assert os.getcwd()=='/content/darknet_for_colab/data', 'Directory should be "/content/darknet_for_colab/data" instead of "{}"'.format(os.getcwd())

# download custom data of weapons from drive
!cp /content/drive/MyDrive/Weapon_Detection/FINAL2-Updated.zip .
!unzip FINAL2-Updated.zip
!rm FINAL2-Updated.zip

In [11]:
!mkdir ts
!cp -r images/. ts/
!cp -r yolo-labels/. ts/
!rm -r images
!rm -r yolo-labels

In [12]:
#add your own class names here
#I am adding only 'pistol' in the class.names file. If you have more than one 
#classes, add each class name in the new line. 
!echo pistol > classes.names

In [13]:
#creating train.txt and test.txt files
!echo > train.txt
!echo > test.txt

Creating Training and Valdiation sets

In [ ]:
import random
images=[]
for file1 in os.listdir('/content/darknet_for_colab/data/ts/'):
  if file1.endswith('.jpg'):
    images.append(file1)

#putting 1000 images in validation/test set
val=random.sample(images,1000)
train=[n for n in images if n not in val]

file_object = open('train.txt', 'w')
for i in train:
  path='/content/darknet_for_colab/data/ts/'+i
  file_object.write(path+'\n')
file_object.close()

file_object = open('test.txt', 'w')
for i in val:
  path='/content/darknet_for_colab/data/ts/'+i
  file_object.write(path+'\n')
file_object.close()

## Visualize custom dataset examples (optional)

In [15]:
%cd ..

/content/darknet_for_colab


In [ ]:
assert os.getcwd()=='/content/darknet_for_colab', 'Directory should be "/content/darknet_for_colab" instead of "{}"'.format(os.getcwd())

import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import glob

def read_label(image_path):
  file_name = image_path.replace('.jpg', '.txt')
  with open(file_name, 'rt') as file:
    print(os.path.basename(file_name) + ': \n' + file.read())

image_path = glob.glob("data/ts/*.jpg")
fig = plt.figure(figsize=(12,8))
cols = 2
rows = 2
grid = gridspec.GridSpec(nrows=rows, ncols=cols, figure=fig)
for i in range(cols*rows):
  fig.add_subplot(grid[i])
  image=plt.imread(image_path[i])
  plt.title(os.path.basename(image_path[i]))
  plt.axis(False)
  plt.imshow(image)
  read_label(image_path[i])

plt.savefig("dataset_examples.jpg", dpi=300)

## Modify yolov4 architecture

**Double click on file `yolov4_config.py` to modify the hyperpameters directly from Colab environment**

E.g: I will train my dataset with these parameters:
 - classes= 1, 
 - max_batches=8000
 - batch=64
 - subdivisions=16
 - width=416
 - height=416
 - ...

In [17]:
assert os.getcwd()=='/content/darknet_for_colab', 'Directory should be "/content/darknet_for_colab" instead of "{}"'.format(os.getcwd())

# Run python script to create our customize yolov4_custom_train.cfg 
# and yolov4_custom_tes.cfg in folder /cfg
!python yolov4_setup.py

[INFO] Generating yolov4_custom_train.cfg successfully...
[INFO] Generating yolov4_custom_test.cfg successfully...


## Training (When no previous checkpoints available)

In [ ]:
assert os.getcwd()=='/content/darknet_for_colab', 'Directory should be "/content/darknet_for_colab" instead of "{}"'.format(os.getcwd())

!./darknet detector train data/yolov4.data cfg/yolov4_custom_train.cfg yolov4.conv.137 -dont_show -map
#If you get CUDA out of memory adjust subdivisions above!
#adjust max batches down for shorter training above

# Resuming from a checkpoint

In [ ]:
assert os.getcwd()=='/content/darknet_for_colab', 'Directory should be "/content/darknet_for_colab" instead of "{}"'.format(os.getcwd())

!./darknet detector train data/yolov4.data cfg/yolov4_custom_train.cfg /content/drive/MyDrive/Weapon_Detection/Yolov4_backup/yolov4_custom_train_last.weights -dont_show -map

## Visualize training result

In [ ]:
assert os.getcwd()=='/content/darknet_for_colab', 'Directory should be "/content/darknet_for_colab" instead of "{}"'.format(os.getcwd())

# Plotting training result after 2000 epochs
import matplotlib.pyplot as plt
fig = plt.figure(figsize=(10,10))
train_result = plt.imread("chart.png")
plt.axis(False)
plt.imshow(train_result)

In [ ]:
# show labels of our custom traffic sign dataset
assert os.getcwd()=='/content/darknet_for_colab', 'Directory should be "/content/darknet_for_colab" instead of "{}"'.format(os.getcwd())

!cat data/classes.names

## Test with example image/video

In [ ]:
assert os.getcwd()=='/content/darknet_for_colab', 'Directory should be "/content/darknet_for_colab" instead of "{}"'.format(os.getcwd())

# download example images and video
!wget --no-check-certificate "https://onedrive.live.com/download?cid=A86CBC7F31A1C06B&resid=A86CBC7F31A1C06B%21120&authkey=AM5VslNNW9a8aO8" -O examples.zip
!unzip examples.zip
!rm -r examples.zip

In [ ]:
assert os.getcwd()=='/content/darknet_for_colab', 'Directory should be "/content/darknet_for_colab" instead of "{}"'.format(os.getcwd())

# cfg/coco.data was harcode in darknet, thus we need 
# to duplicate one with the same content of data/yolov4.data
%cp data/yolov4.data cfg/coco.data

In [ ]:
assert os.getcwd()=='/content/darknet_for_colab', 'Directory should be "/content/darknet_for_colab" instead of "{}"'.format(os.getcwd())

# test out our detector!
# coco.names is hardcoded somewhere in the detector
img_path = "examples/test_image_4.jpg"
!./darknet detect cfg/yolov4_custom_test.cfg backup/yolov4_custom_train_last.weights {img_path} -dont-show

In [ ]:
assert os.getcwd()=='/content/darknet_for_colab', 'Directory should be "/content/darknet_for_colab" instead of "{}"'.format(os.getcwd())

import matplotlib.pyplot as plt 
fig = plt.figure(figsize=(12,12))
plt.axis(False)
processed_image = plt.imread("./predictions.jpg")
plt.imshow(processed_image)

In [ ]:
assert os.getcwd()=='/content/darknet_for_colab', 'Directory should be "/content/darknet_for_colab" instead of "{}"'.format(os.getcwd())

# video processed
!python darknet_video.py -v examples/test_video.mp4 -c cfg/yolov4_custom_test.cfg -w backup/yolov4_custom_train_last.weights -o output.mp4

## The End!